In [4]:
from typing import List
from langchain_core.documents import Document
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_core.runnables import RunnableConfig

from qdrant_client import QdrantClient, models
import logging

from langchain_qdrant import QdrantVectorStore

import os
import requests

# Initialize LangSmith project
os.environ["LANGSMITH_PROJECT"] = 'tg-bot'

QDRANT_URL = os.getenv("QDRANT_URL")
LLM_API_SERVER_URL = os.getenv("LLM_API_SERVER_URL")
LLM_MODEL_NAME = os.getenv("LLM_MODEL_NAME")

logging.info("Modules and Environment variables loaded.")
logging.info("Initializing Qdrant client.")

# Initialize Qdrant client
client_qd = QdrantClient(url=QDRANT_URL)

logging.info("Qdrant client initialized.")

# emb_model_name = '/models/multilingual-e5-large-instruct'
# embeddings = HuggingFaceEmbeddings(model_name=emb_model_name)

logging.info("Initializing embeddings.")

emb_model_name = 'nomic-embed-text'

embeddings = OllamaEmbeddings(
    base_url=LLM_API_SERVER_URL,
    model=emb_model_name
)

logging.info(f"Loaded embeddings model: {emb_model_name}")


2025-08-12 23:06:56,965 [INFO] Modules and Environment variables loaded.
2025-08-12 23:06:56,966 [INFO] Initializing Qdrant client.
2025-08-12 23:06:57,038 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-08-12 23:06:57,040 [INFO] Qdrant client initialized.
2025-08-12 23:06:57,040 [INFO] Initializing embeddings.
2025-08-12 23:06:57,108 [INFO] Loaded embeddings model: nomic-embed-text


In [5]:
# vectorstore initialization

collection_name = 'recall_memories'

recall_memories =  QdrantVectorStore(
        client=client_qd,
        collection_name=collection_name,
        embedding=embeddings,
        vector_name="vector")


2025-08-12 23:07:05,961 [INFO] HTTP Request: GET http://localhost:6333/collections/recall_memories "HTTP/1.1 200 OK"
2025-08-12 23:07:06,262 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [62]:


def get_user_thread_id(config: RunnableConfig) -> str:
    user_id = config["configurable"].get("user_id")
    thread_id = config["configurable"].get("thread_id")
    if user_id is None:
        raise ValueError("User ID needs to be provided to save a memory.")
    if thread_id is None:
        raise ValueError("Thread ID needs to be provided to save a memory.")

    return user_id, thread_id



def save_recall_memories(
        memory: str,
        config: RunnableConfig       
        ) -> str:
    """Save recall memory to vectorstore for later semantic retrieval."""
    user_id, thread_id = get_user_thread_id(config)
    document = Document(
        page_content=memory,
        metadata={"user_id": user_id, 'thread_id': thread_id}
    )

    print(document)

    recall_memories.add_documents([document])
    return memory



def search_recall_memories(
        query: str,
        config: RunnableConfig
) -> List[str]:
    """Search for relevant recall memories."""

    user_id, thread_id = get_user_thread_id(config)
    
    # Filter by user_id and thread_id
    qdrant_filter = models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.user_id",
                match=models.MatchValue(value=user_id),
            ),
            models.FieldCondition(
                key="metadata.thread_id",
                match=models.MatchValue(value=thread_id),
            ),
        ]
    )

    documents = recall_memories.similarity_search(
        query,
        k=3,
        filter=qdrant_filter,  # structured filter required by QdrantVectorStore
    )

    return [doc.page_content for doc in documents]


In [44]:
# load from  backend
user_id = "user_123"
thread_id = "thread_456"


config = {
    "configurable": {
        "user_id": user_id,
        "thread_id": thread_id
    }
}


In [45]:
save_recall_memories("погода", config=config)

page_content='погода' metadata={'user_id': 'user_123', 'thread_id': 'thread_456'}


'погода'

In [63]:

query = "погода"

search_recall_memories(query,config=config)

['погода']

In [29]:
get_user_thread_id(config)

('user_123', 'thread_456')

In [40]:
texts = ["тест", "пример текста"]
vectors = embeddings.embed_documents(texts)
print("Embedding vectors shape:", len(vectors), "x", len(vectors[0]))


Embedding vectors shape: 2 x 768


In [ ]:
collection_name = 'recall_memories'


def search_recall_memories(
        query: str,
        config: RunnableConfig
) -> List[str]:
    """Search for relevant recall memories."""

    user_id, thread_id = get_user_thread_id(config)
    
    # Filter by user_id and thread_id
    qdrant_filter = models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.user_id",
                match=models.MatchValue(value=user_id),
            ),
            models.FieldCondition(
                key="metadata.thread_id",
                match=models.MatchValue(value=thread_id),
            ),
        ]
    )

    documents = recall_memories.similarity_search(
        query,
        k=3,
        filter=qdrant_filter,  # structured filter required by QdrantVectorStore
    )

    return [doc.page_content for doc in documents]

([Record(id='a2767bf6-eb6e-4a6a-a23b-5da19f352575', payload={'page_content': 'погода', 'metadata': {'user_id': 'user_123', 'thread_id': 'thread_456'}}, vector=None, shard_key=None, order_value=None)],
 None)